In [1]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd

# In Anaconda -> Environments -> Not Installed -> pandas-datareader -> Apply
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

# pip install cufflinks -> In Qt Console
import cufflinks as cf
# pip install plotly==5.3.1 -> In Qt Console
import plotly.express as px
import plotly.graph_objects as go

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

import warnings
warnings.simplefilter("ignore")

# pip install yfinance in Qt Console or Powershell Prompt on Windows
import yfinance as yf

from plotly.subplots import make_subplots

In [2]:
# Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
msft = yf.download(tickers='MSFT', period='1mo', interval='5m')
msft

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-01-19 09:30:00-05:00,306.290009,307.049988,305.299988,305.940002,305.940002,1830264
2022-01-19 09:35:00-05:00,305.890015,310.100006,305.609985,309.779999,309.779999,1345963
2022-01-19 09:40:00-05:00,309.799988,309.959991,308.709991,309.734894,309.734894,1012492
2022-01-19 09:45:00-05:00,309.730011,312.119995,309.380005,312.059906,312.059906,1551396
2022-01-19 09:50:00-05:00,312.029999,313.279999,311.739990,312.640015,312.640015,1469688
...,...,...,...,...,...,...
2022-02-18 15:35:00-05:00,287.290009,287.869995,287.170013,287.589996,287.589996,352512
2022-02-18 15:40:00-05:00,287.600006,288.720001,287.440002,288.510010,288.510010,483401
2022-02-18 15:45:00-05:00,288.529999,289.049988,287.987000,288.500000,288.500000,472198


## Plot a Candlestick Chart

In [ ]:
x = msft.index
close = msft['Adj Close']
high = msft['High']
low = msft['Low']
openp = msft['Open']

# Calculates 5 and 20 day moving average
# Takes prices and divides them by a defined time period
# Short periods react quickly, while longer react slower to price changes
# They are used to see if a stock is in an uptrend
# If a short term trend is above the longterm that is a sign an uptrend is expected
msft['MA5'] = msft.Close.rolling(5).mean()
msft['MA20'] = msft.Close.rolling(20).mean()

# Create candlestick plot
candles = go.Candlestick(x=x, open=openp, high=high,
                         low=low, close=close, name="Candles")

# Create 5 and 20 day moving average
ma5 = go.Scatter(x=msft.index, y=msft.MA5, 
                 line=dict(color='orange', width=1), name="MA5")
ma20 = go.Scatter(x=msft.index, y=msft.MA20, 
                  line=dict(color='green', width=1), name="MA20")

# Create volume bar chart
vol = go.Bar(x=msft.index, y=msft['Volume'], name="Volume")

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add plots
fig.add_trace(trace=candles, secondary_y=True)
fig.add_trace(trace=ma5, secondary_y=True)
fig.add_trace(trace=ma20, secondary_y=True)
fig.add_trace(trace=vol, secondary_y=False)


# Valid Steps : 'month', 'year', 'day', 'hour', 'minute', 'second', 'all'
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=15,
                     label="15M",
                     step="minute",
                     stepmode="backward"),
                dict(count=45,
                     label="45M",
                     step="minute",
                     stepmode="backward"),
                dict(count=1,
                     label="1HR",
                     step="hour",
                     stepmode="todate"),
                dict(count=1,
                     label="1D",
                     step="day",
                     stepmode="todate"),
                dict(count=3,
                     label="3D",
                     step="day",
                     stepmode="todate"),
                dict(count=7,
                     label="1W",
                     step="day",
                     stepmode="backward"),
                dict(count=14,
                     label="2W",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1M",
                     step="month",
                     stepmode="backward"),
                dict(label="All", step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)
fig.update_layout(xaxis_title="Dates", yaxis_title="Stock Price", 
                  title="Microsoft Candlestick Chart", 
                  width=1000, height=800)

# Hides plot between saturday and monday
# Hide hours between 9:30am to 4pm
# Hide specific days like holidays
fig.update_xaxes(
        rangeslider_visible=True,
        rangebreaks=[
            dict(bounds=["sat", "mon"]),
            dict(bounds=[16, 9.5], pattern="hour"), 
            dict(values=["2020-12-25", "2021-01-01", "2021-07-04"])
        ]
    )

fig.show()

## Download Multiple Stocks

In [ ]:
stocks = ["MKC", "NEM", "ODFL"]
stocks_df = yf.download(tickers=stocks, period='1mo', interval='5m')
stocks_df

## Simple Moving Average
A SMA allows you to see the big picture when analyzing a stock. It however it takes time to catch up to current trends. This lag for a 100 day moving average would be 100/2 or 50 days. Long term moving averages can be extremely important as we'll see when we look at the Death Cross and Golden Cross.


In [ ]:
# Download MSFT, NEM and the S&P
msft_df = yf.download(tickers='MSFT', period='10y', interval='1d')
msft_df

In [ ]:
nem_df = yf.download(tickers='NEM', period='10y', interval='1d')
nem_df

In [ ]:
gspc_df = yf.download(tickers='^gspc', period='10y', interval='1d')
gspc_df

## Calculate 20 Day Moving Averages

In [ ]:
msft_ma20 = msft_df['Adj Close'].rolling(window=20).mean()
msft_ma20.tail()

In [ ]:
nem_ma20 = nem_df['Adj Close'].rolling(window=20).mean()
nem_ma20.tail()

In [ ]:
gspc_ma20 = gspc_df['Adj Close'].rolling(window=20).mean()
gspc_ma20.tail()

## Calculate 100 Day Moving Average

In [ ]:
# Allows us to study the averaged out long term trends
msft_ma100 = msft_df['Adj Close'].rolling(window=100).mean()
nem_ma100 = nem_df['Adj Close'].rolling(window=100).mean()
gspc_ma100 = gspc_df['Adj Close'].rolling(window=100).mean()
gspc_ma100.tail()

## Plot MSFT with Moving Averages

In [ ]:
ma20 = go.Scatter(x=msft_ma20.index, y=msft_ma20, 
                  line=dict(color='orange', width=1), name="MA20")
ma100 = go.Scatter(x=msft_ma100.index, y=msft_ma100, 
                   line=dict(color='green', width=1), name="MA100")
msft_prc = go.Scatter(x=msft_df.index, y=msft_df['Adj Close'], 
                      line=dict(color='blue', width=1), name="Price")

fig = go.Figure()
fig.add_trace(ma20)
fig.add_trace(ma100)
fig.add_trace(msft_prc)
                                              
fig.update_xaxes(
    rangeslider_visible=True, title='Zoom on Dates Using Slider')
fig.update_yaxes(title="Stock Price (USD)")
fig.show()

## Exponential Moving Average
A EMA can be used to reduce the lag by putting more emphasis on recent price data.


In [ ]:
# Calculate MSFTs 20 day EMA
msft_ema20 = msft_df['Adj Close'].ewm(span=20, adjust=False).mean()

ema20 = go.Scatter(x=msft_ema20.index, y=msft_ema20, 
                   line=dict(color='green', width=1), name="EMA20")
ma20 = go.Scatter(x=msft_ma20.index, y=msft_ma20, 
                  line=dict(color='orange', width=1), name="MA20")
msft_prc = go.Scatter(x=msft_df.index, y=msft_df['Adj Close'], 
                      line=dict(color='blue', width=1), name="Price")

fig = go.Figure()
fig.add_trace(ma20)
fig.add_trace(ema20)
fig.add_trace(msft_prc)
                                              
fig.update_xaxes(
    rangeslider_visible=True, title='Zoom on Dates Using Slider')
fig.update_yaxes(title="Stock Price (USD)")
fig.show()

# As you can see a trading stategy where we buy when the price 
# hits the moving average from below signals a buy and vice versa.

## Death and Golden Crosses
When a Death Cross occurs, that is a sign that a major sell off will occur. A Death Cross is said to occur typically when the 50 day moving average falls below a 200 day. A Golden Cross accures when the short term average crosses the long term again moving higher.



In [ ]:
gspc_df = yf.download(tickers='^gspc', period='max', interval='1d')
gspc_df

In [ ]:
gspc_ma50 = gspc_df['Adj Close'].rolling(window=50).mean()
gspc_ma200 = gspc_df['Adj Close'].rolling(window=200).mean()

## Plot S&P

In [ ]:
ma50 = go.Scatter(x=gspc_ma50.index, y=gspc_ma50, 
                  line=dict(color='orange', width=1), name="MA50")
ma200 = go.Scatter(x=gspc_ma200.index, y=gspc_ma200, 
                   line=dict(color='green', width=1), name="MA200")
gspc_prc = go.Scatter(x=gspc_df.index, y=gspc_df['Adj Close'], 
                      line=dict(color='blue', width=1), name="Price")

fig = go.Figure()
fig.add_trace(ma50)
fig.add_trace(ma200)
fig.add_trace(gspc_prc)
                                              
fig.update_xaxes(
    rangeslider_visible=True, title='Zoom on Dates Using Slider')
fig.update_yaxes(title="Stock Price (USD)")
fig.show()

# You can see many examples of death & golden crosses below

## Cufflinks Quant Figure Module

In [ ]:
# add_sma - Simple Moving Average
# add_ema() - Exponential Moving Average
# add_trendline() - Trend line
# add_support() - Support line
# add_rsi - Relative Strength Indicator
# add_adx - Average Directional Index
# add_atr() - Average True Range
# add_macd() - Moving Average Convergence Divergence
# add_bollinger_bands - Bollinger Bands
# add_volume - Volume bars
# add_cci() - Commodity Channel Indicator
# add_dmi() - Directional Movement Index
# add_ptps() - Parabolic SAR
# add_resistance() - Resistance line


qf=cf.QuantFig(gspc_df, title='S&P 500 Data',legend='top',name='GS')
qf.add_bollinger_bands()
qf.add_volume()

qf.iplot()